# Prototype Voorspellend model PC6 Data

### Samenvatting:
X, oftewel de onafhankelijke variabelen. Dit worden de demografische gegevens van het CBS.
Er is minimaal één set nodig maar het mogen er meer zijn. Hoe veel data er minimaal nodig is en of één model past bij heel Nederland past zijn open vragen. Het kan altijd nauwkeuriger, is het devies. Een fractie van X wordt gebruikt om de nauwkeurigheid van het model te scoren. Als het model bepaalde groepen beter begrijpt dan anderen is de score erg variabel. 
De datasets mogen GEEN foutieve waarden bevatten. Gebruik Pandas om te controleren of er outliers of NaN's aanwezig zijn. Missende waarden kunnen het beste worden opgevuld met een mediaan of gemiddelde. Gebruik daarvoor een waarde van een overkoepelende zone of bepaal deze zelf. Het is mogelijk om deze waarde te voorspellen op basis van kolommen die wel helemaal goed ingevuld zijn.

y, oftewel de afhankelijke variabele. Dat is hier de stemuitslag per ???? of PC6, per partij plus de opkomst. Uiteraard moet bij elk verkiezingsjaar een dataset met het juiste jaartal komen. Het model kan alleen leren van data waarvan vaststaat dat elke Xn hoort bij elke Yn.

Het model, een lineare regressie in de laatste versie. Met dit model kunnen we nagaan welke kolommen veel invloed hebben op de beslissing die het model neemt. Ook is het mogelijk om per voorspellen een foutmarge te berekenen.

Ten slotte gebruiken we een X waarvan we nog geen y hebben om de y te voorspellen.

Opmerkingen:
Omdat de data van het CBS niet precies op de verkiezingsdag is afgenomen zal de berekening van de opkomst niet tot op de komma nauwkeurig zijn.
Het maakt voor het model niet uit of getallen komen van een gemiddelde van een kleine buurt of een grote gemeenten.
Hoe nauwkeuriger en fijnmaziger de data, hoe beter het model verbanden kan ontdekken. Naarmate de zones kleiner worden, moet de data over stemgedrag ook even nauwkeurig uit die zone gehaald worden. Houd er rekening mee dat modellen zelden goed omgaan met uitschieters.
De gerichte GL campagne beperkt zich tot 20 gemeenten in de lijst in deze map.

To Do:
Maak een model voor ELK Schaalniveau. Als je dan gegevens mist van een buurt of straat gebruik je de data of voorspelling van een wijk. Als de data van een wijk mist gebruik je de data van een gemeente, maar dat zal niet precies genoeg zijn voor het beoogde doel.

### Databases en links:
Hoe deelt het CBS data in op basis van postcode?
https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/gegevens-per-postcode
Informatie over data uit 2019 in Odata4 (Wijken en buurten)
https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer\
Het campagneteam heeft toegang tot data op PC6 niveau, voor meer info vraag het aan Jasper.
In de oude github staan de uitslagen uit 2017

In [1]:
# import database
url2019 = "https://odata4.cbs.nl/CBS/84583NED" # Kerncijfers wijken en buurten 2019
url2017 = "https://odata4.cbs.nl/CBS/83765NED" # Kerncijfers wijken en buurten 2017
# Toelichting: https://www.cbs.nl/nl-nl/maatwerk/2017/31/kerncijfers-wijken-en-buurten-2017

### Checklist:
Welke X gaan we gebruiken? De laatste Europarlementsverkiezingen vonden plaats in mei 2019. De laatste tweedekamerverkiezingen vonden plaats in maart 2017. Ook zijn er nog de Provinciale Statenverkiezingen.
We hebben toegang tot demografische data op PC6-niveau afkomstig van het CBS. Dat moet wel steeds uit het jaartal zijn dat overeenkomt met de y.
Welke y gaan we gebruiken? We hebben toegang tot stemuitslagen op PC6-niveau.
Is er nog foutieve data aanwezig?
Weten we zeker dat de stemuitslagen worden ingedeeld op basis van dezelfde zones? Zijn de Vonoroy-diagrammen (met correcties) goed genoeg?

De meeste features in de CBS data zijn absoluut, maar niet allemaal! Alle features moeten genormaliseerd zijn,
anders kan het model anders kijken naar een groot aantal inwoners zonder te relativeren.

## Imports, laden en controleren data

In [2]:
import numpy as np
import pandas as pd
import re
import time
import CBSparserOData4
from PC4e6_database import *
#importeer van scikit learn alleen de delenfuncties/klassen die nodig zijn.
from sklearn import linear_model, model_selection
from sklearn.multioutput import MultiOutputRegressor

### Info over features
Zie PC4e6_database

### Kies features en laad de data

In [3]:
stemTK, register = laad_uitslagenPC6("adressen_gl_prioriteit/adressen_TK2017.csv")

C:\Users\DKuipers\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [4]:
demo = laad_CBSdataPC6('CBS_PC6_selectie.csv')

## Data filteren en combineren

In [18]:
dataPC6 = join_PC6(stemTK,demo)
dataPC6.head()

,nummer,lat,lon,stemperc,prioriteit,totaal,opkomst,D66_stemmen,GL_stemmen,PvdA_stemmen,...,INW_2544,INW_4564,INW_65PL,AANTAL_HH,GEM_HH_GR,WONING,WON_MRGEZ,UITKMINAOW,Perc_NW_migracht,Gemcode2019
PC6,,,,,,,,,,,,,,,,,,,,,
1011AB,111.0,52.377773,4.905684,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,...,5.0,0.0,0.0,5.0,1.4,0.0,0.0,0.0,NaN,363.0
1011AC,147.0,52.377385,4.908563,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,...,10.0,0.0,5.0,20.0,1.5,5.0,5.0,0.0,20.0,363.0
1011AD,1.0,52.376820,4.904492,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,363.0
1011AE,145.0,52.376399,4.910231,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,363.0
1011AG,98.5,52.375865,4.902948,0.773567,0.0,1082.0,0.814146,226.0,290.0,133.0,...,5.0,0.0,0.0,5.0,1.6,0.0,0.0,0.0,NaN,363.0


In [19]:
feature_select = ['MAN','VROUW','UITKMINAOW','GEM_HH_GR']
# 'WON_MRGEZ' delen door aantal huishoudens levert niet een fractie van aantal huishoudens(waarde tussen 0 en 1) op.
# Gebruik deze feature pas als we weten wat het betekent.
dataPC6 = normaliseer_PC6(dataPC6,feature_select, drempel = 20)
dataPC6.describe()

['MAN', 'VROUW', 'INW_014', 'INW_1524', 'INW_2544', 'INW_4564', 'INW_65PL', 'UITKMINAOW', 'GEM_HH_GR', 'stemperc', 'D66_percentage', 'GL_percentage', 'PvdA_percentage', 'PvdD_percentage', 'SP_percentage', 'DENK_percentage']


,MAN,VROUW,INW_014,INW_1524,INW_2544,INW_4564,INW_65PL,UITKMINAOW,GEM_HH_GR,stemperc,D66_percentage,GL_percentage,PvdA_percentage,PvdD_percentage,SP_percentage,DENK_percentage
count,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000,120007.000000
mean,0.494014,0.505679,0.121963,0.098546,0.277413,0.247877,0.135222,0.069653,2.107513,0.531592,0.151304,0.134044,0.067525,0.043247,0.089244,0.046210
std,0.091277,0.091346,0.120471,0.130793,0.174171,0.138824,0.184661,0.112442,0.615959,0.120775,0.056953,0.057718,0.020762,0.017951,0.039143,0.077059
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.062459,0.016916,0.011711,0.000000,0.000000,0.005731,0.000000
25%,0.437500,0.444444,0.000000,0.000000,0.166667,0.166667,0.000000,0.000000,1.600000,0.443107,0.108682,0.092099,0.052898,0.030082,0.059764,0.004604
50%,0.500000,0.500000,0.125000,0.066667,0.272727,0.250000,0.076923,0.000000,2.100000,0.517457,0.139104,0.116788,0.065106,0.040000,0.084546,0.016349
75%,0.548387,0.560000,0.214286,0.166667,0.400000,0.333333,0.200000,0.125000,2.500000,0.620502,0.187342,0.167553,0.078958,0.053172,0.114850,0.051109
max,1.000000,1.000000,0.800000,1.000000,1.000000,0.888889,1.000000,1.000000,6.000000,0.912863,0.395252,0.556017,0.184240,0.128571,0.256579,0.602515


### Lineare regressie

In [20]:
ycols = ['stemperc']
xcols = feature_select
for col in dataPC6.columns:
    if re.search('percentage$',col):
        ycols.append(col)
    elif re.search('^INW',col):
        xcols.append(col)
print(ycols, '\n', xcols)
# Schud de data en maak het model! Noteer ook hoe je foutmarges weergeeft. Wat is de beste verdeling van train/test?

ycols = ['GL_percentage']

Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(dataPC6[xcols], dataPC6[ycols], test_size=0.12, random_state=14)

['stemperc', 'D66_percentage', 'GL_percentage', 'PvdA_percentage', 'PvdD_percentage', 'SP_percentage', 'DENK_percentage'] 
 ['MAN', 'VROUW', 'UITKMINAOW', 'GEM_HH_GR', 'INW_014', 'INW_1524', 'INW_2544', 'INW_4564', 'INW_65PL']


In [21]:
prototype = linear_model.LinearRegression()
#prototype = MultiOutputRegressor(linear_model.LinearRegression())
prototype.fit(Xtrain,ytrain)
accuracy = prototype.score(Xtest,ytest)
print(accuracy)

0.17304317803258584


Score op enkel 'GL_percentage, geen drempel, feature_select = ['MAN','VROUW','UITKMINAOW'] = 0.1273
Score op enkel 'GL_percentage, drempel = 10, feature_select = ['MAN','VROUW','UITKMINAOW'] = 0.1827
Score op enkel 'GL_percentage, drempel = 20, feature_select = ['MAN','VROUW','UITKMINAOW'] = 0.1727

In [9]:
help(ycols.remove)

Help on built-in function remove:

remove(...) method of builtins.list instance
    L.remove(value) -> None -- remove first occurrence of value.
    Raises ValueError if the value is not present.



In [10]:
lijst_gemeenten = open("lijst_gemeenten.txt")
doelen_gemeenten = []
for regel in lijst_gemeenten:
    doelen_gemeenten.append(regel.strip("\n "))
print(doelen_gemeenten)

['Alkmaar', 'Arnhem', 'Deventer', 'Enschede', 'Maastricht', 'Wageningen', 'Almere', 'Breda', 'Diemen', 'Groningen', 'Nijmegen', 'Zaanstad', 'Amsterdam', 'Delft', 'Drechtsteden', 'Haarlem', 'Rotterdam', 'Zeist', 'Amstelveen', 'Den Bosch', 'Ede', 'Leeuwarden', 'Tilburg', 'Zutphen', 'Apeldoorn', 'Den Haag', 'Eindhoven', 'Leiden', 'Utrecht', 'Zwolle']
